### 3-D Borehole Plots

Not all boreholes are perfectly vertical. With well-logging equipment the Vermont Geological Survey and SUNY Plattsburgh are able to quantify a borehole's deviation from vertical and the location of the borehole in x, y, z space. The following script interprets the information from the well-logging and plots the well in a 3-D interactive view.

In [1]:
# Import necessary libraries

import pandas as pd
import numpy as np
import math
import plotly
import plotly.graph_objs as go

ModuleNotFoundError: No module named 'plotly'

The script reads in excel files with well-logging information contained in the following three columns:

- 'Depth (m)'	
- 'Azimuth (°)'	
- 'Tilt (°)'

In [ ]:
# Read in data to dataframe

bore_data = pd.read_excel("")
print('This dataset contains {0} rows.'.format(len(bore_data)))

### Calculating the position of well-bore in X, Y, Z space

The starting point of the well is set to X = 0 and Y = 0. The initial True Vertical Depth is set to the first depth measurement.
- X increases to the east and decreases to the west. 
- Y increases to the north and decreases to the south.



In [ ]:
# Set Initial Well Position to Starting Depth, X = 0, Y = 0

bore_data.loc[0,'True Vertical Depth (m)'] = bore_data.loc[0,'Depth (m)']
bore_data.loc[0,'X (m)'] = 0
bore_data.loc[0,'Y (m)'] = 0

##### Calculation Methods
From the Depth, Azimuth, and Tilt measurements from the well-logging equipment we can calculate the position of the well in x, y, z space. Depending on the required accuracy and interval between measurements there are a series of [calculation methods](https://wiki.aapg.org/Depth_and_thickness_conversion).

In the case of these wells the interval between measurements is 0.004 meters. This is such a short interval the borehole can be accurately described by linear segments between points. For a larger interval the Minimum Curvature Method is more accurate.

In [ ]:
# Iterate through dataframe and calculate True Vertical Depth, X, and Y for each set of borehole measurements

for i in range(1, len(raw_borehole_data)):
    bore_data.loc[i,'True Vertical Depth (m)'] = (bore_data.loc[(i - 1), 'True Vertical Depth (m)'] + ((bore_data.loc[i, 'Depth (m)'] - bore_data.loc[(i-1), 'Depth (m)'])*math.cos(math.radians(bore_data.loc[i, 'Tilt (°)']))))
    bore_data.loc[i, 'X (m)'] = (bore_data.loc[(i-1), 'X (m)']) + ((bore_data.loc[i, 'Depth (m)'] - bore_data.loc[(i-1), 'Depth (m)'])*(math.sin(math.radians(bore_data.loc[i, 'Tilt (°)'])))*(math.cos(math.radians(bore_data.loc[i, 'Azimuth (°)']))))
    bore_data.loc[i, 'Y (m)'] = (bore_data.loc[(i-1), 'Y (m)']) + ((bore_data.loc[i, 'Depth (m)'] - bore_data.loc[(i-1), 'Depth (m)'])*(math.sin(math.radians(bore_data.loc[i, 'Tilt (°)'])))*(math.sin(math.radians(bore_data.loc[i, 'Azimuth (°)']))))

In [ ]:
# Set data for plot and express depth as negative numbers

x = bore_data['X (m)']
y = bore_data['Y (m)']
z = bore_data['True Vertical Depth (m)'] * -1

Use [Plotly](https://plot.ly/python/) for its ability to create 3-D interactive [line plots](https://plot.ly/python/3d-line-plots/)

In [ ]:
# Configure Plotly to be rendered inline in the notebook

plotly.offline.init_notebook_mode()

# Configure the trace

trace = go.Scatter3d(
    x=x,  
    y=y,  
    z=z,  
    mode='markers',
    marker={
        'size': 2,
        'opacity': 0.8,
        'color' : bore_data['Tilt (°)'],
        'colorbar' : dict(title='Tilt (°)'),
        'colorscale' : 'JET'
    }
)

# Configure the layout
layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0},
    scene=dict(camera=dict(eye=dict(x=4, y=4, z=4)), #the default values are 1.25, 1.25, 1.25
           xaxis=dict(),
           yaxis=dict(),
           zaxis=dict(),
           aspectmode='manual', # this can be 'data', 'cube', 'auto', 'manual'
           # custom aspect ratio
           aspectratio=dict(x=1, y=1, z=5)
           )
)

data = [trace]

plot_figure = go.Figure(data=data, layout=layout)

# Render the plot.
plotly.offline.iplot(plot_figure)